In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)
from scipy.misc import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib 
%matplotlib inline



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [29]:
root_dir = os.getcwd()
img_dir = [os.path.join(root_dir, 'train'),os.path.join(root_dir, 'test')]

pixels = np.array(['pixel_{:03d}'.format(x) for x in range(784)])

for i in range(0,2):
    for char_name in sorted(os.listdir(img_dir[i])):
        char_dir = os.path.join(img_dir[i], char_name)
        img_df = pd.DataFrame(columns=pixels)

        for img_file in sorted(os.listdir(char_dir)):
            image = imread(os.path.join(char_dir, img_file))
            image = image[2:30,2:30]
            image = pd.Series(image.flatten(),index = pixels)
            img_df = img_df.append(image.T, ignore_index=True)

        img_df = img_df.astype(np.uint8)
        img_df['character'] = char_name
        
        if(i==0):
            img_df.to_csv('data.csv', index=False, mode='a', header=flag)
        else:
            img_df.to_csv('data_test.csv', index=False, mode='a', header=flag)        

In [30]:
df = pd.read_csv('data.csv')

df['character_class'] = LabelEncoder().fit_transform(df.character)
df.drop('character', axis=1, inplace=True)
df = df.astype(np.uint8)

df_test = pd.read_csv('data_test.csv')

df_test['character_class'] = LabelEncoder().fit_transform(df_test.character)
df_test.drop('character', axis=1, inplace=True)
df_test = df_test.astype(np.uint8)

# Dimension of df is: [78200 x 1025]
# 1700  =  (85%) of 2000 (each class has 2000 samples)
# 78200 =  1700*46 characters 
# 46    =  number of classes
# 1025  = 32*32 = 1024 feature columns, one output column

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
cor = df.corr()

In [39]:

cor_target = abs(cor.iloc[784])
cor_target = cor_target.to_numpy()
cor_target = cor_target[0:783]
# plt.figure()
# plt.title('Correlation of features with output class')
# plt.xlabel('Pixel Number')
# plt.ylabel('Correlation')
# plt.plot(cor_target)
# plt.savefig('correlation.png')
to_delete = []
for i,j in enumerate(cor_target):
    if(j<0.009):
        to_delete.append(i)
# print(to_delete)
relevant_features = cor_target[cor_target>0.009]
# print(relevant_features.shape)

In [38]:
dataset = df.to_numpy()
dataset_test = df_test.to_numpy()
dataset = np.delete(dataset, to_delete,axis=1)
dataset_test = np.delete(dataset_test,to_delete,axis =1)
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

np.random.shuffle(dataset)
np.random.shuffle(dataset_test)
# print(dataset)
# print(dataset.shape)
X_train = dataset[0:10000,:-1]
Y_train = dataset[0:10000, -1]
# X_validation = dataset[2000:2820,:-1]
# Y_validation = dataset[2000:2820, -1]
X_test  = dataset_test[0:5000,:-1]
Y_test  = dataset_test[0:5000, -1]




(13800, 724)


In [35]:
def classifier(X_train,Y_train, X_test, Y_test, kernel_type, decFuncShape):
    print(kernel_type," kernel", " & ", decFuncShape , " classification method" )
#     kf = KFold(n_splits = 5, shuffle = False)
#     fold_count = 1
#     for train_index, validation_index in kf.split(X_train):
#         print("Fold : ", fold_count)
#         X_train_fold, X_validation_fold = X_train[train_index], X_train[validation_index]
#         Y_train_fold, Y_validation_fold = Y_train[train_index], Y_train[validation_index]

    if(kernel_type == 'rbf'):
        clf = svm.SVC(kernel = kernel_type, gamma = 'scale', C=1,decision_function_shape=decFuncShape,probability = True).fit(X_train, Y_train)
        joblib.dump(clf, 'RBF_10K.pkl')
    elif(kernel_type == 'poly'):
        clf = svm.SVC(kernel = kernel_type, degree = 2, C=1,decision_function_shape=decFuncShape,probability = True).fit(X_train, Y_train)
        joblib.dump(clf, 'POLY_10K.pkl')
    elif(kernel_type == 'linear'):
        clf = svm.SVC(kernel = kernel_type, C=1,decision_function_shape=decFuncShape, probability = True).fit(X_train, Y_train)
        joblib.dump(clf, 'LINEAR_10K.pkl')
    else:
        print("Invalid kernel")
 
    y_pred = clf.predict(X_test)
    acc = accuracy_score(Y_test, y_pred) * 100
    print("Accuracy : ", acc , "%")
    
#         fold_count+=1

In [36]:
if __name__ == '__main__':
    #SVM WITH NO KERNEL
#     classifier(X_train,Y_train,X_test,Y_test,'linear','ovr')  #ONE-VS-ALL CLASSIFICATION METHOD
# #     classifier(X_train,Y_train,X_validation,Y_validation,'linear','ovo')  #ONE-VS-ONE CLASSIFICATION METHOD

#     #SVM WITH RBF KERNEL
#     classifier(X_train,Y_train,X_test,Y_test,'rbf','ovr')     #ONE-VS-ALL CLASSIFICATION METHOD
# #     classifier(X_train,Y_train,X_validation,Y_validation,'rbf','ovo')     #ONE-VS-ONE CLASSIFICATION METHOD

#     #SVM WITH QUADRATIC POLYNOMIAL KERNEL
    classifier(X_train,Y_train,X_test,Y_test,'poly','ovr')    #ONE-VS-ALL CLASSIFICATION METHOD
#     classifier(X_train,Y_train,X_test,Y_test,'poly','ovo')    #ONE-VS-ONE CLASSIFICATION METHOD

poly  kernel  &  ovr  classification method
Accuracy :  82.92 %
